## ETL: Silver para Gold
### Transformacao de Dados Integrados para Modelo Dimensional

Objetivo: Carregar dados da camada Silver (silver.tb_partidas_completa) e transforma-los em um modelo dimensional otimizado para analises de negocio.

Arquitetura:
- Schema: gold (separado do silver)
- Modelo: Star Schema (1 Fato + 4 Dimensoes)
- Nomenclatura: Padrao Mnemonico Corporativo

## 1. Configuracao e Importacoes

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'brasileirao',
    'user': 'postgres',
    'password': 'postgres'
}

CONNECTION_STRING = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(CONNECTION_STRING, pool_size=10, max_overflow=20, echo=False)

print("Configuracao carregada")
print(f"Database: {DB_CONFIG['database']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}")
print(f"Origem: silver.tb_partidas_completa")
print(f"Destino: gold schema (Star Schema)")

## 2. Extracao de Dados da Camada Silver

In [ ]:
query_silver = """
SELECT * FROM silver.tb_partidas_completa
ORDER BY partida_id
"""

print("Extraindo dados da camada Silver...")
df_silver = pd.read_sql(query_silver, engine)

print(f"Registros extraidos: {len(df_silver):,}")
print(f"Colunas: {len(df_silver.columns)}")
print(f"Periodo: {df_silver['data'].min()} ate {df_silver['data'].max()}")
df_silver.head()

## 3. Funcoes Auxiliares

In [ ]:
def mapear_regiao(estado):
    if pd.isna(estado):
        return None
    
    regioes = {
        'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
        'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
        'Centro-Oeste': ['DF', 'GO', 'MT', 'MS'],
        'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
        'Sul': ['PR', 'RS', 'SC']
    }
    
    for regiao, estados in regioes.items():
        if estado.upper() in estados:
            return regiao
    return None

def calcular_kpis(row):
    taxa_conv_mandante = (row['mandante_placar'] / row['mandante_chutes'] * 100) if row['mandante_chutes'] > 0 else 0
    taxa_conv_visitante = (row['visitante_placar'] / row['visitante_chutes'] * 100) if row['visitante_chutes'] > 0 else 0
    efic_mandante = (row['mandante_chutes_alvo'] / row['mandante_chutes'] * 100) if row['mandante_chutes'] > 0 else 0
    efic_visitante = (row['visitante_chutes_alvo'] / row['visitante_chutes'] * 100) if row['visitante_chutes'] > 0 else 0
    
    return pd.Series({
        'vl_par_man_tax_con': round(taxa_conv_mandante, 2),
        'vl_par_vis_tax_con': round(taxa_conv_visitante, 2),
        'vl_par_man_efi': round(efic_mandante, 2),
        'vl_par_vis_efi': round(efic_visitante, 2)
    })

print("Funcoes auxiliares carregadas")

## 4. Criacao da Dimensao TEMPO

In [ ]:
datas_unicas = df_silver[['data', 'hora']].drop_duplicates()
datas_unicas['data'] = pd.to_datetime(datas_unicas['data'])

dim_tempo_list = []
for idx, row in datas_unicas.iterrows():
    data = row['data']
    hora = row['hora'] if pd.notna(row['hora']) else None
    
    dim_tempo_list.append({
        'dt_tem_dat': data,
        'nr_tem_ano': data.year,
        'nr_tem_mes': data.month,
        'ds_tem_mes_nom': data.strftime('%B'),
        'nr_tem_tri': (data.month - 1) // 3 + 1,
        'nr_tem_sem': 1 if data.month <= 6 else 2,
        'nr_tem_dia_mes': data.day,
        'nr_tem_dia_ano': data.timetuple().tm_yday,
        'nr_tem_dia_sem': data.isoweekday(),
        'ds_tem_dia_sem': data.strftime('%A'),
        'fl_tem_fim_sem': data.isoweekday() in [6, 7],
        'ds_tem_tmp': f"{data.year}/{data.year + 1}" if data.month >= 5 else f"{data.year - 1}/{data.year}",
        'hr_tem_hor': hora
    })

dim_tempo = pd.DataFrame(dim_tempo_list)
dim_tempo = dim_tempo.sort_values('dt_tem_dat').reset_index(drop=True)

print(f"DIM_TEM_TEMPO criada: {len(dim_tempo):,} registros")
print(f"Periodo: {dim_tempo['dt_tem_dat'].min()} ate {dim_tempo['dt_tem_dat'].max()}")
dim_tempo.head()

## 5. Criacao da Dimensao TIME

In [ ]:
times_mandantes = df_silver[['mandante', 'mandante_estado']].rename(columns={
    'mandante': 'nk_tim_tim', 
    'mandante_estado': 'ds_tim_est'
})

times_visitantes = df_silver[['visitante', 'visitante_estado']].rename(columns={
    'visitante': 'nk_tim_tim', 
    'visitante_estado': 'ds_tim_est'
})

dim_time = pd.concat([times_mandantes, times_visitantes], ignore_index=True)
dim_time = dim_time.drop_duplicates(subset=['nk_tim_tim']).reset_index(drop=True)

dim_time['ds_tim_nom'] = dim_time['nk_tim_tim']
dim_time['ds_tim_reg'] = dim_time['ds_tim_est'].apply(mapear_regiao)
dim_time['fl_tim_pri_div'] = True

dim_time = dim_time[dim_time['nk_tim_tim'].notna()].copy()
dim_time = dim_time[dim_time['nk_tim_tim'].str.strip() != ''].copy()
dim_time = dim_time[['nk_tim_tim', 'ds_tim_nom', 'ds_tim_est', 'ds_tim_reg', 'fl_tim_pri_div']]

print(f"DIM_TIM_TIME criada: {len(dim_time):,} times")
print("Times por regiao:")
print(dim_time['ds_tim_reg'].value_counts())
dim_time.head(10)

## 6. Criacao da Dimensao ARENA

In [ ]:
dim_arena = df_silver[['arena', 'estado_partida']].copy()
dim_arena = dim_arena.rename(columns={
    'arena': 'nk_are_are',
    'estado_partida': 'ds_are_est'
})

dim_arena = dim_arena.drop_duplicates(subset=['nk_are_are']).reset_index(drop=True)
dim_arena = dim_arena[dim_arena['nk_are_are'].notna()].copy()
dim_arena = dim_arena[dim_arena['nk_are_are'].str.strip() != ''].copy()

dim_arena['ds_are_nom'] = dim_arena['nk_are_are']
dim_arena['ds_are_reg'] = dim_arena['ds_are_est'].apply(mapear_regiao)
dim_arena['qt_are_cap'] = None
dim_arena['ds_are_tip_gra'] = None

dim_arena = dim_arena[['nk_are_are', 'ds_are_nom', 'ds_are_est', 'ds_are_reg', 'qt_are_cap', 'ds_are_tip_gra']]

print(f"DIM_ARE_ARENA criada: {len(dim_arena):,} estadios")
print("Arenas por regiao:")
print(dim_arena['ds_are_reg'].value_counts())
dim_arena.head(10)

## 7. Criacao da Dimensao RESULTADO

In [ ]:
dim_resultado = df_silver[[
    'vencedor', 'tipo_resultado', 'foi_equilibrado', 
    'foi_goleada', 'teve_virada', 'categoria_gols'
]].copy()

dim_resultado = dim_resultado.rename(columns={
    'vencedor': 'ds_res_ven',
    'tipo_resultado': 'ds_res_tip_res',
    'foi_equilibrado': 'fl_res_equ',
    'foi_goleada': 'fl_res_gol',
    'teve_virada': 'fl_res_vir',
    'categoria_gols': 'ds_res_cat_gol'
})

dim_resultado = dim_resultado.drop_duplicates().reset_index(drop=True)
dim_resultado['ds_res_ven'] = dim_resultado['ds_res_ven'].fillna('-')

print(f"DIM_RES_RESULTADO criada: {len(dim_resultado):,} combinacoes")
print("Distribuicao por tipo de resultado:")
print(dim_resultado['ds_res_tip_res'].value_counts())
dim_resultado.head(10)

## 8. Carregar Dimensoes no Banco de Dados

In [ ]:
print("Carregando dimensoes no schema gold...")

with engine.connect() as conn:
    conn.execute(text("TRUNCATE TABLE gold.fct_par_partida CASCADE"))
    conn.execute(text("TRUNCATE TABLE gold.dim_tem_tempo RESTART IDENTITY CASCADE"))
    conn.execute(text("TRUNCATE TABLE gold.dim_tim_time RESTART IDENTITY CASCADE"))
    conn.execute(text("TRUNCATE TABLE gold.dim_are_arena RESTART IDENTITY CASCADE"))
    conn.execute(text("TRUNCATE TABLE gold.dim_res_resultado RESTART IDENTITY CASCADE"))
    conn.commit()
    print("Tabelas limpas")

dim_tempo.to_sql('dim_tem_tempo', engine, schema='gold', if_exists='append', index=False)
print(f"dim_tem_tempo: {len(dim_tempo):,} registros")

dim_time.to_sql('dim_tim_time', engine, schema='gold', if_exists='append', index=False)
print(f"dim_tim_time: {len(dim_time):,} registros")

dim_arena.to_sql('dim_are_arena', engine, schema='gold', if_exists='append', index=False)
print(f"dim_are_arena: {len(dim_arena):,} registros")

dim_resultado.to_sql('dim_res_resultado', engine, schema='gold', if_exists='append', index=False)
print(f"dim_res_resultado: {len(dim_resultado):,} registros")

print("Todas as dimensoes carregadas com sucesso")

## 9. Buscar Surrogate Keys das Dimensoes

In [ ]:
print("Buscando Surrogate Keys das dimensoes...")

dim_tempo_srk = pd.read_sql("SELECT srk_tem_tem, dt_tem_dat, hr_tem_hor FROM gold.dim_tem_tempo", engine)
dim_tempo_srk['dt_tem_dat'] = pd.to_datetime(dim_tempo_srk['dt_tem_dat'])

dim_time_srk = pd.read_sql("SELECT srk_tim_tim, nk_tim_tim, ds_tim_nom FROM gold.dim_tim_time", engine)

dim_arena_srk = pd.read_sql("SELECT srk_are_are, nk_are_are, ds_are_nom FROM gold.dim_are_arena", engine)

dim_resultado_srk = pd.read_sql(
    """SELECT srk_res_res, ds_res_ven, ds_res_tip_res, fl_res_equ, fl_res_gol, fl_res_vir, ds_res_cat_gol 
       FROM gold.dim_res_resultado""", 
    engine
)

print(f"dim_tem_tempo SRKs: {len(dim_tempo_srk):,}")
print(f"dim_tim_time SRKs: {len(dim_time_srk):,}")
print(f"dim_are_arena SRKs: {len(dim_arena_srk):,}")
print(f"dim_res_resultado SRKs: {len(dim_resultado_srk):,}")

## 10. Preparar Tabela Fato com Chaves Estrangeiras

In [ ]:
print("Construindo tabela fato com chaves estrangeiras...")

fato = df_silver.copy()
fato['data'] = pd.to_datetime(fato['data'])

fato = fato.merge(
    dim_tempo_srk[['srk_tem_tem', 'dt_tem_dat']], 
    left_on='data',
    right_on='dt_tem_dat',
    how='left'
).drop('dt_tem_dat', axis=1)

fato = fato.merge(
    dim_time_srk[['srk_tim_tim', 'nk_tim_tim']].rename(columns={
        'srk_tim_tim': 'srk_tim_man', 
        'nk_tim_tim': 'mandante'
    }),
    on='mandante',
    how='left'
)

fato = fato.merge(
    dim_time_srk[['srk_tim_tim', 'nk_tim_tim']].rename(columns={
        'srk_tim_tim': 'srk_tim_vis', 
        'nk_tim_tim': 'visitante'
    }),
    on='visitante',
    how='left'
)

fato = fato.merge(
    dim_arena_srk[['srk_are_are', 'nk_are_are']].rename(columns={'nk_are_are': 'arena'}),
    on='arena',
    how='left'
)

fato['vencedor_join'] = fato['vencedor'].fillna('-')

fato = fato.merge(
    dim_resultado_srk.rename(columns={'ds_res_ven': 'vencedor_join'}),
    on=['vencedor_join', 'ds_res_tip_res', 'fl_res_equ', 'fl_res_gol', 'fl_res_vir', 'ds_res_cat_gol'],
    how='left'
).drop('vencedor_join', axis=1)

print(f"Joins realizados")
print(f"Registros: {len(fato):,}")

## 11. Calcular KPIs e Preparar Colunas Finais

In [ ]:
print("Calculando KPIs e preparando estrutura final...")

kpis = fato.apply(calcular_kpis, axis=1)
fato = pd.concat([fato, kpis], axis=1)

fato_final = pd.DataFrame({
    'nk_par_par': fato['partida_id'],
    'srk_tim_man': fato['srk_tim_man'],
    'srk_tim_vis': fato['srk_tim_vis'],
    'srk_tem_tem': fato['srk_tem_tem'],
    'srk_are_are': fato['srk_are_are'],
    'srk_res_res': fato['srk_res_res'],
    'nr_par_rod': fato['rodata'],
    'qt_par_gol_man': fato['mandante_placar'],
    'qt_par_gol_vis': fato['visitante_placar'],
    'qt_par_dif_gol': fato['diferenca_gols'],
    'qt_par_tot_gol': fato['total_gols'],
    'qt_par_man_chu': fato['mandante_chutes'],
    'qt_par_man_chu_alv': fato['mandante_chutes_alvo'],
    'vl_par_man_pos_bol': fato['mandante_posse_bola'],
    'qt_par_man_pas': fato['mandante_passes'],
    'vl_par_man_pre_pas': fato['mandante_precisao_passes'],
    'qt_par_man_fal': fato['mandante_faltas'],
    'qt_par_man_car_ama': fato['mandante_cartoes_amarelos'],
    'qt_par_man_car_ver': fato['mandante_cartoes_vermelhos'],
    'qt_par_man_imp': fato['mandante_impedimentos'],
    'qt_par_man_esc': fato['mandante_escanteios'],
    'qt_par_vis_chu': fato['visitante_chutes'],
    'qt_par_vis_chu_alv': fato['visitante_chutes_alvo'],
    'vl_par_vis_pos_bol': fato['visitante_posse_bola'],
    'qt_par_vis_pas': fato['visitante_passes'],
    'vl_par_vis_pre_pas': fato['visitante_precisao_passes'],
    'qt_par_vis_fal': fato['visitante_faltas'],
    'qt_par_vis_car_ama': fato['visitante_cartoes_amarelos'],
    'qt_par_vis_car_ver': fato['visitante_cartoes_vermelhos'],
    'qt_par_vis_imp': fato['visitante_impedimentos'],
    'qt_par_vis_esc': fato['visitante_escanteios'],
    'qt_par_tot_gol_man': fato['total_gols_mandante'],
    'qt_par_tot_gol_vis': fato['total_gols_visitante'],
    'qt_par_tot_gol_par': fato['total_gols_partida'],
    'qt_par_gol_con': fato['gols_contra'],
    'qt_par_gol_pen': fato['gols_penalty'],
    'qt_par_tot_car_ama': fato['total_cartoes_amarelos'],
    'qt_par_tot_car_ver': fato['total_cartoes_vermelhos'],
    'qt_par_tot_car': fato['total_cartoes'],
    'vl_par_man_tax_con': fato['vl_par_man_tax_con'],
    'vl_par_vis_tax_con': fato['vl_par_vis_tax_con'],
    'vl_par_man_efi': fato['vl_par_man_efi'],
    'vl_par_vis_efi': fato['vl_par_vis_efi']
})

print(f"KPIs calculados")
print(f"Colunas na tabela fato: {len(fato_final.columns)}")
fato_final.head()

## 12. Carregar Tabela Fato no Banco de Dados

In [ ]:
print("Carregando tabela fato no schema gold...")

batch_size = 1000
total_batches = len(fato_final) // batch_size + 1

for i in range(0, len(fato_final), batch_size):
    batch = fato_final.iloc[i:i+batch_size]
    batch.to_sql('fct_par_partida', engine, schema='gold', if_exists='append', index=False)
    current_batch = i // batch_size + 1
    print(f"Batch {current_batch}/{total_batches} carregado ({len(batch)} registros)")

print(f"Tabela fato carregada com sucesso")
print(f"Total de registros: {len(fato_final):,}")

## 13. Validacao dos Dados Carregados

In [ ]:
print("Validando dados carregados no schema gold...")

with engine.connect() as conn:
    count_tempo = conn.execute(text("SELECT COUNT(*) FROM gold.dim_tem_tempo")).scalar()
    count_time = conn.execute(text("SELECT COUNT(*) FROM gold.dim_tim_time")).scalar()
    count_arena = conn.execute(text("SELECT COUNT(*) FROM gold.dim_are_arena")).scalar()
    count_resultado = conn.execute(text("SELECT COUNT(*) FROM gold.dim_res_resultado")).scalar()
    count_fato = conn.execute(text("SELECT COUNT(*) FROM gold.fct_par_partida")).scalar()
    
    print("DIMENSOES:")
    print(f"  dim_tem_tempo:      {count_tempo:,} registros")
    print(f"  dim_tim_time:       {count_time:,} registros")
    print(f"  dim_are_arena:      {count_arena:,} registros")
    print(f"  dim_res_resultado:  {count_resultado:,} registros")
    print("FATO:")
    print(f"  fct_par_partida:    {count_fato:,} registros")
    
    count_view = conn.execute(text("SELECT COUNT(*) FROM gold.vw_partidas_completas")).scalar()
    print("VIEWS:")
    print(f"  vw_partidas_completas: {count_view:,} registros")
    
    print("Amostra da view vw_partidas_completas:")
    df_view = pd.read_sql(
        """SELECT dt_tem_dat, ds_mandante, ds_visitante, 
                  qt_par_gol_man, qt_par_gol_vis, ds_vencedor 
           FROM gold.vw_partidas_completas 
           LIMIT 5""", 
        conn
    )
    print(df_view)

print("Validacao concluida com sucesso")

## 14. Resumo da Execucao

In [ ]:
print("ETL SILVER PARA GOLD CONCLUIDO COM SUCESSO")
print("")
print("RESUMO:")
print(f"  Schema: gold (separado do silver)")
print(f"  Modelo: Star Schema (1 Fato + 4 Dimensoes)")
print(f"  Nomenclatura: Padrao Mnemonico Corporativo")
print(f"  Dimensoes populadas: 4")
print(f"  Tabela Fato populada: 1")
print(f"  Views criadas: 2")
print("")
print("VOLUMES:")
print(f"  Partidas analisadas: {count_fato:,}")
print(f"  Times: {count_time:,}")
print(f"  Arenas: {count_arena:,}")
print(f"  Resultados: {count_resultado:,}")
print(f"  Periodos: {count_tempo:,}")
print("")
print("NOMENCLATURA APLICADA:")
print("  srk = Surrogate Key (chave substituta)")
print("  nk  = Natural Key (chave natural)")
print("  ds  = Description (descricao/texto)")
print("  qt  = Quantity (quantidade inteira)")
print("  vl  = Value (valor decimal)")
print("  fl  = Flag (booleano)")
print("  dt  = Date (data)")
print("  nr  = Number (numero)")
print("  hr  = Hour (hora)")